In [ ]:
# default_exp cikFInfo

# cikFInfo

> Save parsed form info for each CIK in a separate JSON-format file.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

import collections
import json
import os

from secscan import utils, dailyList
from secscan import scrape13F, scrape8K, scrape6K, scrape13G, scrape13D, scrape4

defaultCikFInfoDir = os.path.join(utils.stockDataRoot,'cikFInfo')
allScraperClasses = [scrape13F.scraper13F,scrape8K.scraper8K,scrape6K.scraper6K,
                     scrape13G.scraper13G,scrape13D.scraper13D,scrape4.scraper4]
cikFPrefLen = 4

Save parsed form info for each CIK in a separate JSON-format file.
The format is designed so that info for additional forms can simply be appended to the file.

In [ ]:
#export

def getCikFInfoDirAndPath(cik, cikFInfoDir=defaultCikFInfoDir) :
    if len(cik)<2 or not cik.isdigit() or cik[0]=='0' :
        raise ValueError(f'invalid CIK "{cik}"')
    fDir = os.path.join(cikFInfoDir,cik[:cikFPrefLen])
    return fDir,os.path.join(fDir,cik+'.json')

def jsonValError(msg, s) :
    if len(s) > 200 :
        s = s[:100] + ' ... ' + s[-100:]
    return ValueError(msg + ' in ' + s)

def loadCikFInfo(cik, cikFInfoDir=defaultCikFInfoDir, returnAsText=False) :
    cik = str(cik).lstrip('0')
    fPath = getCikFInfoDirAndPath(cik, cikFInfoDir)[1]
    if not os.path.exists(fPath) :
        return {}
    with open(fPath,'r',encoding='ascii') as f :
        s = f.read().strip()
    if s[-1] != ',' :
        raise jsonValError('missing ending ,', s)
    if returnAsText :
        return s[:-1]
    return json.loads('{'+s[:-1]+'}')

def saveCikFInfo(cik, cikFInfo, removeDups=False, cikFInfoDir=defaultCikFInfoDir) :
    if removeDups :
        existingCikFInfo = loadCikFInfo(cik, cikFInfoDir=cikFInfoDir)
        cikFInfo = dict((k,v) for k,v in cikFInfo.items() if k not in existingCikFInfo)
    if len(cikFInfo) == 0 :
        return
    s = json.dumps(cikFInfo, indent=0).strip()
    if s[0]!='{' or s[-1]!='}' :
        raise jsonValError('missing start/end {}', s)
    fDir, fPath = getCikFInfoDirAndPath(cik, cikFInfoDir)
    if not os.path.exists(fDir) :
        os.makedirs(fDir)
    with open(fPath,'a',encoding='ascii') as f :
        f.write(s[1:-1])
        f.write(',\n')

def saveAllCikFInfo(startD, endD, scraperClasses,
                    removeDups=True, cikFInfoDir=defaultCikFInfoDir, ciks=None) :
    dl = dailyList.dailyList(startD=startD, endD=endD)
    datesPresent = utils.loadPklFromDir(cikFInfoDir, "dates.pkl", set())
    cikInfoMap = {}
    for scraperClass in scraperClasses :
        scraper = scraperClass(startD=startD, endD=endD)
        scraper.addToCikInfoMap(dl, cikInfoMap, ciks=ciks, excludeDates=datesPresent)
    for cik,cikFInfo in cikInfoMap.items() :
        if (ciks is not None and cik not in ciks) :
            continue
        saveCikFInfo(cik, cikFInfo, removeDups=removeDups, cikFInfoDir=cikFInfoDir)
    datesPresent.update(dl.dl.keys())
    utils.savePklToDir(cikFInfoDir, "dates.pkl", datesPresent)

def saveYears(startY, endY,
              removeDups=False, cikFInfoDir=defaultCikFInfoDir, ciks=None) :
    qList = ['0101', '0401', '0701', '1001', '0101']
    for y in range(startY, endY) :
        for qs, qe in zip(qList, qList[1:]) :
            startD, endD = f'{y}{qs}', f'{y+1 if qe=="0101" else y}{qe}'
            print(startD, endD)
            saveAllCikFInfo(startD, endD, allScraperClasses,
                            removeDups=removeDups, cikFInfoDir=cikFInfoDir, ciks=ciks)

Code to check CIK format and figure out the right prefix length:

In [ ]:
# def checkCiks() :
#     cikNames = utils.pickLoad(os.path.join(utils.stockDataRoot,'dlMaps','cikNames.pkl'))
#     print('ciks with leading 0', [cik for cik in cikNames if cik[0]=='0'])
#     print('less than 4 long', [cik for cik in cikNames if len(cik)<4])
#     for prefLen in [3,4] :
#         ciksByPref = collections.defaultdict(list)
#         for cik in cikNames :
#             ciksByPref[cik[:prefLen]].append(cik)
#         print(f'prefix length {prefLen}: {len(ciksByPref)} folders,'
#               +f' max {max(len(v) for v in ciksByPref.values())} files')
# checkCiks()
# OUTPUT: 
# ciks with leading 0 []
# less than 4 long ['63']
# prefix length 3: 773 folders, max 9154 files
# prefix length 4: 4003 folders, max 933 files
# - chose prefix length 4

In [ ]:
htmlPref = """
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<style>
.collapsible {
  background-color: #DDD;
  cursor: pointer;
  padding: 2px;
  margin: 2px;
  border: 1px solid black;
  text-align: left;
  outline: none;
  font-size: 15px;
}
.collapsible:hover {
  background-color: #AAA;
}
.content {
  padding: 0 2px;
  display: none;
  overflow: hidden;
  background-color: #DDD;
}
.bodyclass {
  padding: 0 2px;
  font-size: 16px;
  font-family: Arial;
}
</style>
</head>
<body class="bodyclass">
<script>
"""

htmlSuff = """
var fList = submissions['filings']['recent'];
var accNos = fList['accessionNumber'];
var forms = fList['form'];
var fDates = fList['filingDate'];
var lastFDate = '';
var secUrlPref = 'https://www.sec.gov';
function urlForAccNo(accNo) {
    return secUrlPref + "/Archives/edgar/data/"
            + accNo.replace("-","") + "/" + accNo + "-index.htm";
}
function urlForCik(cik) {
    return secUrlPref + "/cgi-bin/browse-edgar?CIK=" + cik;
}
function showItem(urlText, url, text, textLimit1, textLimit2) {
    var res='', textLimit=textLimit2;
    if (urlText != '') {
        res = "<a href='" + secUrlPref + url + "' target='_blank'>"
                + urlText + "</a>";
        textLimit = textLimit1;
    }
    return res + " &nbsp;<button type='button' class='collapsible'>"
                        + text.slice(0,textLimit) + "</button>"
                    + "<div class='content'>" + text + "</div>";
}
function form4F() {
    res = ' ***4***';
    return res;
}
function form8KF(info) {
    var urlText=info['links'][0][0], url=info['links'][0][3];
    var res = '';
    if ('explanatoryNote' in info) {
        res += ' &nbsp; ' + showItem(urlText, url, info['explanatoryNote'], 100, 50);
        urlText = '';
    }
    var itemTexts = info['itemTexts'];
    for (let i in itemTexts) {
        if (itemTexts.length>1 && itemTexts[i].toLowerCase().startsWith('item 9'))
            continue;
        res += ' &nbsp; ' + showItem(urlText, url, itemTexts[i], 100, 50);
        urlText = '';
    }
    var text99s=info['text99'], links=info['links'];
    for (let i in text99s)
        if (text99s[i] != '')
            res += ' &nbsp; ' + showItem(links[i][0], links[i][3], text99s[i], 100, 50);
    return res;
}
var formFuncs = {
    '4': form4F,
    '8-K': form8KF
};
function htmlFor(form, accNo) {
    var res = "<a href='" + urlForAccNo(accNo) + "' target='_blank'>"
                + form + "</a>";
    if (form.endsWith('/A'))
        form = form.slice(0,-2);
    if (!(form in formFuncs))
        return res;
    if (accNo in fInfo)
        res += formFuncs[form](fInfo[accNo]);
    else
        res += " ---";
    return res;
}
var el = document.createElement("div");
el.innerHTML = "<div><button type='button' id='expandall'>expand all</button>"
                + " <b><a href='" + urlForCik(cik) + "' target='_blank'>"
                        + submissions["name"] + "</a></b></div>";
document.body.appendChild(el);
for (let i in accNos) {
    var el = document.createElement("div");
    formHtml = (lastFDate==fDates[i] ? "&nbsp;&nbsp;&nbsp;&nbsp;" : "<b>"+fDates[i]+"</b>&nbsp; ");
    lastFDate = fDates[i];
    formHtml += htmlFor(forms[i], accNos[i]);
    el.innerHTML = formHtml;
    document.body.appendChild(el);
}
var collList = document.getElementsByClassName("collapsible");
for (let i in collList) {
  collList[i].addEventListener("click", function() {
    let content = this.nextElementSibling;
    content.style.display = (content.style.display === "block" ? "none" : "block");
  }
  );
}
document.getElementById("expandall").addEventListener("click", function() {
    let expandingAll = (this.textContent === 'expand all');
    this.textContent = (expandingAll ? 'collapse all' : 'expand all');
    for (let i in collList) {
        let content = collList[i].nextElementSibling;
        content.style.display = (expandingAll ? "block" : "none");
    }
});
</script>
</body>
</html>"""

from secscan import getCikFilings

def makeHtml(cik) :
    cik = str(cik).lstrip('0')
    subs = utils.downloadSecUrl(getCikFilings.cikRestFilingsUrl(cik),
                             restData=True, toFormat='text')
    cikFInfo = loadCikFInfo(cik, returnAsText=True)
    with open(os.path.join(utils.stockDataRoot,cik+'.html'), 'w') as f :
        f.write(htmlPref)
        f.write(f'var cik={cik};')
        f.write('var fInfo={')
        f.write(cikFInfo)
        f.write('};\n')
        f.write('var submissions=')
        f.write(subs)
        f.write(';\n')
        f.write(htmlSuff)

In [ ]:
# makeHtml(732717)

In [ ]:
# l = loadCikFInfo(732717)

In [ ]:
# l['0000732717-23-000007']['itemTexts']

['Item 2.02 Results of Operations and Financial Condition. The registrant announced on January 25, 2023, its results of operations for the fourth quarter of 2022. The text of the press release and accompanying financial information are attached as exhibits and incorporated herein by reference. ',
 'Item 9.01 Financial Statements and Exhibits. The following exhibits are furnished as part of this report: (d) Exhibits 99.1 Press release dated January 25, 2023 reporting financial results for the fourth quarter ended December 31, 2022. 99.2 AT&T Inc. selected financial statements and operating data. 99.3 Discussion and reconciliation of non-GAAP measures. 99.4 Supplemental Quarterly Standalone AT&T Financial Information. 104 Cover Page Interactive Data File (embedded within the Inline XBRL document) Signature Pursuant to the requirements of the Securities Exchange Act of 1934, the Registrant has duly caused this report to be signed on its behalf by the undersigned hereunto duly authorized. 

In [ ]:
# dict((k,v) for k,v in l.items() if k!='filings')

{'cik': '732717',
 'entityType': 'operating',
 'sic': '4813',
 'sicDescription': 'Telephone Communications (No Radiotelephone)',
 'insiderTransactionForOwnerExists': 1,
 'insiderTransactionForIssuerExists': 1,
 'name': 'AT&T INC.',
 'tickers': ['T', 'TBB', 'TBC', 'T-PA', 'T-PC'],
 'exchanges': ['NYSE', 'NYSE', 'NYSE', 'NYSE', 'NYSE'],
 'ein': '431301883',
 'description': '',
 'website': '',
 'investorWebsite': '',
 'category': 'Large accelerated filer',
 'fiscalYearEnd': '1231',
 'stateOfIncorporation': 'DE',
 'stateOfIncorporationDescription': 'DE',
 'addresses': {'mailing': {'street1': '208 S. AKARD ST',
   'street2': None,
   'city': 'DALLAS',
   'stateOrCountry': 'TX',
   'zipCode': '75202',
   'stateOrCountryDescription': 'TX'},
  'business': {'street1': '208 S. AKARD ST',
   'street2': None,
   'city': 'DALLAS',
   'stateOrCountry': 'TX',
   'zipCode': '75202',
   'stateOrCountryDescription': 'TX'}},
 'phone': '2108214105',
 'flags': '',
 'formerNames': [{'name': 'SBC COMMUNICATI

In [ ]:
# saveYears(2017,2024)

20170101 20170401
20170401 20170701
20170701 20171001
20171001 20180101
20180101 20180401
20180401 20180701
20180701 20181001
20181001 20190101
20190101 20190401
20190401 20190701
20190701 20191001
20191001 20200101
20200101 20200401
20200401 20200701
20200701 20201001
20201001 20210101
20210101 20210401
20210401 20210701
20210701 20211001
20211001 20220101
20220101 20220401
20220401 20220701
20220701 20221001
20221001 20230101
20230101 20230401
20230401 20230701
20230701 20231001
20231001 20240101


In [ ]:
# ll = sorted(utils.loadPklFromDir(defaultCikFInfoDir,'dates.pkl','xxx')); ll[0],ll[-1]

('20170101', '20230201')

Test cik file info class:

In [ ]:
assert loadCikFInfo('123')=={}
fDir, fPath = getCikFInfoDirAndPath('12345')
assert fDir.endswith('12345'[:cikFPrefLen]) and fPath.endswith('12345.json')

In [ ]:
#hide
# uncomment and run to regenerate all library Python files
# from nbdev.export import notebook2script; notebook2script()